In [1]:
from music21 import midi, note, chord, stream, scale
from random import randint, uniform

s = stream.Stream()

# optional scale
sc1 = scale.MinorScale('a')
minorScale = sc1.getPitches('c1', 'c#8', direction=scale.Direction.ASCENDING)
print(len(minorScale))

# simplify the syntax
def addFixedNote(pitch, length=1, vel=80):
  newNote = note.Note(pitch, quarterlength=length)
  newNote.volume.velocity = vel
  s.append([newNote])

def addNote(degree, length=1, vel=80):
  if isinstance(degree, list):
    addChord(degree, length, volume)
  else:

    if degree >= len(minorScale):
      degree = len(minorScale) - 1

    if degree < 0:
      degree = 0

    pitch = minorScale[degree]
    newNote = note.Note(pitch)
    newNote.volume.velocity = vel
    newNote.duration.quarterLength = length
    s.append([newNote])


def addChord(degrees, length=1, vel=80):
  notes = [minorScale[d] for d in degrees]
  newChord = chord.Chord(notes)
  newChord.volume.velocity = vel
  newChord.duration.quarterLength = length
  s.append([newChord])


def clip(inVal, minVal, maxVal):
  return max(min(inVal, maxVal), minVal)


50


In [2]:
s = stream.Stream()

for i in range(10):
  pitch = randint(10, 20)
  length = randint(1, 4) * 0.25
  volume = uniform(40, 100)
  addNote(pitch, length, volume)

s.show('midi')

In [3]:
# Chords plus melody

s = stream.Stream()  # clear stream


pitch = 30
volume = 60
for i in range(100):
  pitch += randint(-2, 2)
  if (pitch < 0):
    pitch = 10
  if (pitch >= len(minorScale)):
    pitch = 20

  volume += randint(-5, 5)
  volume = clip(volume, 10, 100)

  if i%7 == 0:
    addChord([pitch, pitch-7, pitch-5], 0.5, 100)
  else:
    addNote(pitch, 0.25, volume)

s.show('midi')

In [ ]:
# Melody with various offsets

s = stream.Stream()  # clear stream

melody = [0, 5, 2, 1]
offset = 10
pitch = 30
volume = 60
for i in range(100):
  pitch = melody[i % 4] + offset

  if i%8 == 0:
    offset = randint(10, 20)

  addNote(pitch, 0.25, 90)

s.show('midi')

In [ ]:
# 3 minute piece with two lines and extra harmonies
# More chord stacks

s = stream.Stream()  # clear stream

melody1 = [0, 5, 2, 1]    # experiment with this
melody2 = [7, 6, 5, 4]    # experiment with this
offset1 = 10
offset2 = 15

pitch = 30
volume = 60
for i in range(200):
  pitch1 = melody1[i % 4] + offset1

  if i%2 == 0:
    addNote(pitch1, 0.25, 90)
  else:
    if randint(0, 100) < 50:
      pitch2 = melody2[i % 4] + offset2
      addNote([pitch1, pitch1+5, pitch2, pitch2+5], 0.5, 90)


  if i%16 == 0:
    offset1 = randint(10, 20)
    offset2 = randint(10, 20)

s.show('midi')

In [ ]:
s = stream.Stream()

t = 0

while t < 10:
  pitch1 = randint(0, len(minorScale))
  pitch2 = pitch1 + 2
  pitch3 = pitch1 + 4
  pitches = [pitch1, pitch2, pitch3]
  print(pitches)
  length = randint(1, 4) * 0.25
  volume = uniform(40, 100)
  addChord(pitches, length, volume)

  t += length

s.show('midi')

[37, 39, 41]
[27, 29, 31]
[17, 19, 21]
[10, 12, 14]
[7, 9, 11]
[39, 41, 43]
[16, 18, 20]
[5, 7, 9]
[35, 37, 39]
[23, 25, 27]
[32, 34, 36]
[41, 43, 45]
[2, 4, 6]
[4, 6, 8]
[13, 15, 17]
[26, 28, 30]
[41, 43, 45]


In [ ]:
s = stream.Stream()     # clear the stream

addFixedNote('C4')
addFixedNote('E4', 0.5)      # optional duration
addFixedNote('G4', 1, 40)    # optional duration and volume (0-100)
s.show('midi')

In [ ]:
s = stream.Stream()

t = 0

while t < 10:
  pitch = randint(60, 72)
  addFixedNote(pitch)
  t += 1

s.show('midi')

In [ ]:
sc1 = scale.MinorScale('a')
sc1.pitches[0]

<music21.pitch.Pitch A4>

In [ ]:
from music21 import converter
melody = converter.parse("tinynotation: 3/4 c4 d8 f g16 a g f#")
melody.show('midi')

In [ ]:
mf = midi.MidiFile()
mt = midi.MidiTrack(1)
me1 = midi.MidiEvent(mt)
me1.type = midi.ChannelVoiceMessages.NOTE_ON
me1.channel = 1
me1.time = 200
me1.pitch = 60
me1.velocity = 120

s = midi.translate.midiFileToStream(mt)
s.show('midi')

AttributeError: 'MidiTrack' object has no attribute 'tracks'

In [ ]:
def playMidi(midiObj):
    # Load MIDI file as a music21 stream
    # s = converter.parse(path)

    # Set up FluidSynth MIDI player
    fs = fluidsynth.Synth()
    # sfid = fl.sfload("/usr/share/sounds/sf2/FluidR3_GM.sf2")
    sfid = fs.sfload("example.sf2")
    fs.program_select(0, sfid, 0, 0)

    # Render the MIDI stream as audio
    audio_data = []
    for event in midiObj.flat.notes:
        fs.noteon(0, event.pitch.midi, int(event.volume.velocity))
        audio_data.extend(fl.get_samples(int(event.duration.quarterLength * 44100)))
        fs.noteoff(0, event.pitch.midi)

    # Normalize audio data and play it in the notebook
    audio_data = np.array(audio_data, dtype="int16")
    audio_data = audio_data / (2**15 - 1)
    return Audio(audio_data, rate=44100)

In [ ]:
me1

<music21.midi.MidiEvent NOTE_ON, t=200, track=1, channel=1, pitch=60, velocity=120>

In [ ]:
playMidi(me1)

AttributeError: module 'fluidsynth' has no attribute 'Synth'

In [ ]:
import time
import fluidsynth

fs = fluidsynth.Synth()
fs.start()

sfid = fs.sfload("example.sf2")
fs.program_select(0, sfid, 0, 0)

fs.noteon(0, 60, 30)
fs.noteon(0, 67, 30)
fs.noteon(0, 76, 30)

time.sleep(1.0)

fs.noteoff(0, 60)
fs.noteoff(0, 67)
fs.noteoff(0, 76)

time.sleep(1.0)

fs.delete()

AttributeError: module 'fluidsynth' has no attribute 'Synth'

In [ ]:
!pip3 show fluidsynth

Name: fluidsynth
Version: 0.2
Summary: Fluidsynth bindings
Home-page: http://corbinsimpson.com/
Author: Corbin Simpson
Author-email: MostAwesomeDude@gmail.com
License: UNKNOWN
Location: /Users/tmudd/Library/Python/3.9/lib/python/site-packages
Requires: 
Required-by: 
